# Mi primera metaclase: un ejemplo práctico

In [1]:
s = '{"iniFecha": "12/05/2010", "imp": 12.24, "nombre": "Don Pimp\xf3n"}'

In [2]:
import json

d = json.loads(s)
d

{'imp': 12.24, 'iniFecha': '12/05/2010', 'nombre': 'Don Pimpón'}

In [3]:
class Model:
    def __init__(self, amount=None, start_date=None, name=None):
        self.amount = amount
        self.start_date = start_date
        self.name = name
        
def from_json(s):
    d = json.loads(s)
    obj = Model()
    obj.amount = d['imp']
    obj.start_date = d['iniFecha']
    obj.name = d['nombre']
    return obj

obj = from_json(s)
obj

Ventajas:
- ¿más *pythonico*?
- snake_case en vez de camelCase
- atributos no claves: `obj.start_date` frente a `d['inifecha']`

Desventajas:
- perdemos representación
- no hay posibilidad de volver a serializar
- mucho código

In [4]:
def to_json(obj):
    d = {}
    d['imp'] = obj.amount
    d['iniFecha'] = obj.start_date
    d['nombre'] = obj.name
    return json.dumps(d)

d2 = to_json(obj)
d2

'{"imp": 12.24, "iniFecha": "12/05/2010", "nombre": "Don Pimp\\u00f3n"}'

In [5]:
class Model:
    def __init__(self, amount=None, start_date=None, name=None):
        self.amount = amount
        self.start_date = start_date
        self.name = name
        
    def __repr__(self):
        return ('Model('
                'amount={s.amount!r}, '
                'start_date={s.start_date!r}, '
                'name={s.name!r})').format(s=self)
    
obj = from_json(s)
obj

Model(amount=12.24, start_date='12/05/2010', name='Don Pimpón')

Desventajas:
- ~~perdemos representación~~
- ~~no hay posibilidad de volver a serializar~~
- MUCHO CÓDIGO

In [6]:
from decimal import Decimal as D
import datetime as dt

def from_json(s):
    d = json.loads(s)
    obj = Model()
    obj.amount = D(str(d['imp']))
    obj.start_date = dt.datetime.strptime(d['iniFecha'], '%d/%m/%Y').date()
    obj.name = d['nombre']
    return obj

obj = from_json(s)
obj

Model(amount=Decimal('12.24'), start_date=datetime.date(2010, 5, 12), name='Don Pimpón')

In [7]:
def to_json(obj):
    d = {}
    d['imp'] = str(obj.amount)
    d['iniFecha'] = obj.start_date.strftime('%d/%m/%Y')
    d['nombre'] = obj.name
    return json.dumps(d)

d3 = to_json(obj)
d3

'{"imp": "12.24", "iniFecha": "12/05/2010", "nombre": "Don Pimp\\u00f3n"}'

Ventajas:
- tipos nativos (decimal, fecha)

Desventajas:
- ¡MUCHO CÓDIGO!

- Cada modelo código repetitivo
- Conversiones desperdigadas

... descriptores al rescate

In [8]:
class Field:
    def __init__(self, name):
        self.name = name
        self.value = None
        
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        else:
            return obj.__dict__[self.name]

    def __set__(self, obj, value):
        self.value = value
        obj.__dict__[self.name] = value

In [12]:
class Model2:
    amount = Field('amount')
    start_date = Field('start_date')
    name = Field('name')
    
    def __init__(self, amount=None, start_date=None, name=None):
        self.amount = amount
        self.start_date = start_date
        self.name = name
        
    def __repr__(self):
        return ('Model2('
                'amount={s.amount!r}, '
                'start_date={s.start_date!r}, '
                'name={s.name!r})').format(s=self)

In [13]:
from decimal import Decimal as D
import datetime as dt

def from_json2(s):
    d = json.loads(s)
    obj = Model2()
    obj.amount = D(str(d['imp']))
    obj.start_date = dt.datetime.strptime(d['iniFecha'], '%d/%m/%Y').date()
    obj.name = d['nombre']
    return obj

obj = from_json2(s)
obj

Model2(amount=Decimal('12.24'), start_date=datetime.date(2010, 5, 12), name='Don Pimpón')

# Conclusiones

Notas

- Los ORM, formularios... utilizan esta técnica.

- No es necesario hacerlo uno mismos: hay librerías que hacen esto. Ver marshmallow o Django REST framework.
